**IMPORTANT NOTE**: Exercie 2 and 3 will only work on your own machine (not via noteable, I think). Also: You'll need to unzip the chrome driver in the folder where your notebook is.

# Exercise 1

Just using beautiful soup go to the description of this course and extract the number of SCQF level for it. http://www.drps.ed.ac.uk/22-23/dpt/cxcmse11427.htm

Spoiler alert: the content should be 'SCQF Level ...'.

*hint*: you can use in to check for occurance of a word in a sentence

In [ ]:
if 'banana' in 'a sentence with bananas':
    print('this sentence has a word banana in it')

# if word in sentence:
#     print('this sentence has a word in it')

In [ ]:
# !pip install bs4 
from urllib.request import urlopen
import os
from bs4 import BeautifulSoup

In [ ]:
# website address
page = 'http://www.drps.ed.ac.uk/22-23/dpt/cxcmse11427.htm'

# open the url and store the website
website = urlopen(page)

# convert the website's content, for this a parser is needed. In this case a html parser
soup = BeautifulSoup(website, 'html.parser')

# Retrieve the cell that contains 'SCQF Level'
# Retrieve the cell that contains 'SCQF Credits'
table = soup.find('table', {'class':'sitstablegrid'})
for cell in table.findChildren('td'):
    if 'SCQF Level' in cell.text:
        print(cell.text)
        break

# Exercise 2

We look at the Google map, Edinburgh castle: https://www.google.com/maps/place/Edinburgh+Castle/@55.9485977,-3.2021022,17z/data=!4m5!3m4!1s0x4887c79a2099c0f7:0x469a1eebe54c0a58!8m2!3d55.9485947!4d-3.1999135

And we try to find all reviews of Edinburgh castle.

In [ ]:
# !pip install selenium 
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# define method that will create a browser, suitable to your operating system
import sys
def get_a_browser():
    if sys.platform.startswith('win32') or sys.platform.startswith('cygwin'):
        return webdriver.Chrome() # windows
    else:
        return webdriver.Chrome('./chromedriver') # mac

In [ ]:
browser = get_a_browser()

# the url we want to open
# fill in the correct url for google map, Edinburgh castle!
# You can also try other places!

url = 'https://www.google.com/maps/place/Edinburgh+Castle/@55.9485947,-3.2021022,17z/data=!3m1!4b1!4m5!3m4!1s0x4887c79a2099c0f7:0x469a1eebe54c0a58!8m2!3d55.9485947!4d-3.1999135'
# the browser will start and load the webpage
browser.get(url)

# we wait 1 second to let the page load everything
time.sleep(1)

# Google map has cookies warning, just accept the cookies

accept_cookies = browser.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button')
print(accept_cookies)
accept_cookies.click();
time.sleep(3)

# Get total number of reviews
total_number_of_reviews = browser.find_element(By.XPATH,
                                               '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').text.split(" ")[0]
# fix the buge, total_number_reviews should include all, not [1:-1]
total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)#Find scroll layout

# go to all reviews page
browser.find_element(By.XPATH,
                     '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]').click()
# some other xpath may work, you can have a try!
time.sleep(3)

In [ ]:
# Scroll down to access all reviews, first of all, let's find the element!
scrollable_div = browser.find_element(By.XPATH,
                                      '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')#Scroll as many times as necessary to load all reviews
"""
Google map reviews display 10 per scoll down, so in total, we need to run round(total_number_of_reviews/10 - 1) times
"""
# for i in range(0,(round(total_number_of_reviews/10 - 1))):
# # However, in order to run fast, we can just load part of all
for i in range(0,100):
    browser.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
            scrollable_div)
    time.sleep(1)

#extract the info
response = BeautifulSoup(browser.page_source, 'html.parser')
reviews = response.find_all('span', class_='wiI7pd', text=True)
dates = response.find_all('span', class_='rsqaWe', text=True)
rates = response.find_all('span', class_="kvMYJc")
users = response.find_all('div', class_='jftiEf fontBodyMedium')

reviews_text=[]
dates_text = []
users_text = []
rates_int = []

for i in range(len(reviews)):
    reviews_text.append(reviews[i].text)
    dates_text.append(dates[i].text)
    rates_int.append(int(rates[i]['aria-label'][1]))
    users_text.append(users[i]['aria-label'])

In [ ]:
#Build up a pandas dataframe to store the dataset
df = pd.DataFrame({"users":users_text, 
                   "rates":rates_int, 
                   "review":reviews_text, 
                   "dates":dates_text})

In [ ]:
df

# Exercise 3

Apply the same code you used before to retrieve all Tweets of President Biden, Barack Obama, or any Tweeter you like

* Find the number of retweets
* Write the scores to a csv file

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
# define method that will create a browser, suitable to your operating system
import sys
def get_a_browser():
    if sys.platform.startswith('win32') or sys.platform.startswith('cygwin'):
        return webdriver.Chrome() # windows
    else:
        return webdriver.Chrome('./chromedriver') # mac

In [ ]:
# launch the browser
browser = get_a_browser()

# launch the page you want to look at
twitter_url = u'https://twitter.com/POTUS'

# Add the search term
query = ''

# Create the url
url = twitter_url+query

# Get the page
browser.get(url)

# Let the Tweets load
time.sleep(1)

# Find the body of the page
body = browser.find_element(By.TAG_NAME,'body')

# Keep scrolling down using a simulation of the PAGE_DOWN button
for _ in range(10):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

# Get the tweets by their class (similar to Beautifulsoup's find({'class':'something'}))
scores = browser.find_elements(By.XPATH,"//div[@data-testid='retweet']");
    
# Write them to csv file
output_file = open('twitter_scores.csv','w')

# Print Tweets
for score in scores:
    if score.text!= "":
        print('Retweets: '+ score.text)
        output_file.write(str(score.text)+"\n")

output_file.close()  